In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
!pip install turicreate

    100% |████████████████████████████████| 86.1MB 487kB/s eta 0:00:01��██▎            | 52.0MB 47.4MB/s eta 0:00:01
    100% |████████████████████████████████| 327kB 37.0MB/s ta 0:00:01
    100% |████████████████████████████████| 2.7MB 14.3MB/s ta 0:00:01
    100% |████████████████████████████████| 23.8MB 1.8MB/s eta 0:00:01
  Running setup.py bdist_wheel for prettytable ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Running setup.py bdist_wheel for resampy ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built prettytable resampy
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

In [4]:
# The code was removed by Watson Studio for sharing.

,customerId,products
0,27270,85123AY
1,27270,71053R
2,27270,84406BH
3,27270,84029GV
4,27270,84029EX


In [5]:

body = client_83b43f68eb9348f8a7795f9fdefa8efd.get_object(Bucket='recommendationsystemforpurchaseda-donotdelete-pr-l0bytgn7jogfsi',Key='test_J1hm2KQ.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()
customers=df_data_2[['CustomerID']]
customers=customers.drop_duplicates('CustomerID')
customers = customers.rename(columns={'CustomerID': 'customerId'})
customers.head()


,customerId
0,127269
1,227268
6,327267
7,162738
16,252747


In [6]:
print(customers.shape)
customers.head()

(628, 1)


,customerId
0,127269
1,227268
6,327267
7,162738
16,252747


In [7]:
print(transactions.shape)
transactions.head()

(330575, 2)


,customerId,products
0,27270,85123AY
1,27270,71053R
2,27270,84406BH
3,27270,84029GV
4,27270,84029EX


## 3. Data preparation
* Our goal here is to break down each list of items in the `products` column into rows and count the number of products bought by a user

In [8]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [i for i in x.split('|')])
transactions.head(10).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0
0,27270,85123AY
1,27270,71053R
2,27270,84406BH
3,27270,84029GV
4,27270,84029EX
5,27270,22752G
6,27270,21730R
7,427266,21756Q
8,462735,22728B
9,462735,22727B


In [10]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,27270,71053R,1
1,27270,85123AY,1


### 3.1. Create data with user, item, and target field
* This table will be an input for our modeling later
    * In this case, our user is `customerId`, `productId`, and `purchase_count`

In [11]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
# data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 1.09 minutes


In [13]:
print(data.shape)
data.head(20)

(271031, 3)


,customerId,productId,purchase_count
0,0,15056BLH,1
1,0,15056ND,1
2,0,15058CH,1
3,0,15060BB,1
4,0,16218Y,1
5,0,16235A,1
6,0,16236S,1
7,0,17091JD,1
8,0,20674R,1
9,0,20675E,1


### 3.2. Create dummy
* Dummy for marking whether a customer bought that item or not.
* If one buys an item, then `purchase_dummy` are marked as 1
* Why create a dummy instead of normalizing it, you ask?
    * Normalizing the purchase count, say by each user, would not work because customers may have different buying frequency don't have the same taste
    * However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [14]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [15]:
data_dummy = create_data_dummy(data)

### 3.3. Normalize item values across users
* To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [20]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head(20)

productId,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,...,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
(df_matrix.shape)

(972, 3810)

In [21]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head(20)

(972, 3810)


productId,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,...,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head(20)

(256976, 3)


,customerId,productId,scaled_purchase_freq
38,25470,10002F,0.0
99,50949,10002F,0.0
114,56340,10002F,0.0
132,64539,10002F,0.0
140,68229,10002F,0.0
197,104499,10002F,0.0
216,120879,10002F,0.0
264,141759,10002F,0.0
268,142758,10002F,0.0
288,153639,10002F,0.0


#### Define a function for normalizing data

In [24]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

## 4. Split train and test set
* Splitting the data into training and testing sets is an important part of evaluating predictive modeling, in this case a collaborative filtering model. Typically, we use a larger portion of the data for training and a smaller portion for testing. 
* We use 80:20 ratio for our train-test set size.
* Our training portion will be used to develop a predictive model, while the other to evaluate the model's performance.
* Now that we have three datasets with purchase counts, purchase dummy, and scaled purchase counts, we would like to split each.

In [25]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(216824, 3) (54207, 3)


In [26]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [27]:
train_data

customerId,productId,purchase_count
224478,23403P,1
329067,22189I,2
60939,21876I,1
105399,35957E,1
326367,82552M,1
372816,22962X,1
273627,20652R,1
216378,21162S,1
181908,85123AY,2
290007,21556T,1


In [28]:
test_data

customerId,productId,purchase_count
452835,22066A,1
111789,79321V,2
308187,85099FC,1
160839,23091E,1
471825,85106U,1
2790,22758C,1
317277,22329R,1
50049,84946F,2
432756,22340Z,1
283707,22379G,1


#### Define a `split_data` function for splitting data to training and test set

In [29]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [30]:
# # lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Baseline Model
Before running a more complicated approach such as collaborative filtering, we would like to use a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity.

### 5.1. Using a Popularity model as a baseline
* The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
* We use `turicreate` library for running and evaluating both baseline and collaborative filtering models below
* Training data is used for model selection

#### Using purchase counts

In [31]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [32]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 216824 observations with 971 users and 3766 items.

Data prepared in: 9.62288s

216824 observations to process; with 3766 unique items.

In [33]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/330575 queries. users per second: 75500.2

recommendations finished on 2000/330575 queries. users per second: 60496.1

recommendations finished on 3000/330575 queries. users per second: 54059.9

recommendations finished on 4000/330575 queries. users per second: 42295.4

recommendations finished on 5000/330575 queries. users per second: 37782

...

recommendations finished on 327000/330575 queries. users per second: 25242.3

recommendations finished on 328000/330575 queries. users per second: 25272.8

recommendations finished on 329000/330575 queries. users per second: 25281.7

recommendations finished on 330000/330575 queries. users per second: 25275.7

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|   27270    |   22423U  | 2.3028322440087146 |  1   |
|   27270    |   20725N  | 2.0897097625329817 |  2   |
|   27270    |   47566Y  | 2.055690072639225  |  3   |
|   27270    |   21196B  |        2.0         |  4   |
|   27270    |  81953BF  |        2.0         |  5   |
|   27270    |   22720A  | 1.9102564102564104 |  6   |
|   27270    |   21212D  | 1.8311345646437995 |  7   |
|   27270    |   23442T  |        1.75        |  8   |
|   27270    |   22960K  | 1.7155172413793103 |  9   |
|   27270    |   47469B  | 1.7142857142857142 |  10  |
|   27270    |   22423U  | 2.3028322440087146 |  1   |
|   27270    |   20725N  | 2.0897097625329817 |  2   |
|   27270    |   47566Y  | 2.055690072639225  |  3   |
|   27270    |   21196B  |        2.0         |  4   |
|   27270    |  81953BF  |        2.0         |  5   |
|   27270 

#### Define a `model` function for model selection

In [34]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [35]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [36]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 968 users and 3757 items.

Data prepared in: 0.823152s

216824 observations to process; with 3757 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   127269   |  90129AJ  |  1.0  |  1   |
|   127269   |   21002B  |  1.0  |  2   |
|   127269   |   21936G  |  1.0  |  3   |
|   127269   |   22554P  |  1.0  |  4   |
|   127269   |   22319Q  |  1.0  |  5   |
|   127269   |   22470E  |  1.0  |  6   |
|   127269   |   90174N  |  1.0  |  7   |
|   127269   |   22666I  |  1.0  |  8   |
|   127269   |   22413V  |  1.0  |  9   |
|   127269   |   22294U  |  1.0  |  10  |
|   227268   |  90129AJ  |  1.0  |  1   |
|   227268   |   21002B  |  1.0  |  2   |
|   227268   |   21936G  |  1.0  |  3   |
|   227268   |   22554P  |  1.0  |  4   |
|   227268   |   22319Q  |  1.0  |  5   |
|   227268   |   22470E  |  1.0  |  6   |
|   227268   |   90174N  |  1.0  |  7   |
|   227268   |   22666I  |  1.0  |  8   |
|   227268   |   22413V  |  1.0  |  9   |
|   227268   |   22294U  |  1.0  |  10  |
|   327267   |  90129AJ  |  1.0  |

#### Using normalized purchase count

In [37]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 205580 observations with 971 users and 2494 items.

Data prepared in: 0.672039s

205580 observations to process; with 2494 unique items.

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|   127269   |   85089Y  |        1.0         |  1   |
|   127269   |  90065AP  |        0.5         |  2   |
|   127269   |   85189T  |        0.5         |  3   |
|   127269   |   84548X  |        0.5         |  4   |
|   127269   |   21816Y  |        0.4         |  5   |
|   127269   |  81950BS  |        0.4         |  6   |
|   127269   |   21018J  | 0.3333333333333333 |  7   |
|   127269   |  84800SH  | 0.3333333333333333 |  8   |
|   127269   |   84358Q  | 0.3333333333333333 |  9   |
|   127269   |  85129CV  | 0.3333333333333333 |  10  |
|   227268   |   85089Y  |        1.0         |  1   |
|   227268   |  90065AP  |        0.5         |  2   |
|   227268   |   85189T  |        0.5         |  3   |
|   227268   |   84548X  |        0.5         |  4   |
|   227268   |   21816Y  |        0.4         |  5   |
|   227268

#### Notes
* Once we created the model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks. 
* In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.
* If a grouping example below, products 132, 248, 37, and 34 are the most popular (best-selling) across customers. Using their purchase counts divided by the number of customers, we see that these products are at least bought 3 times on average in the training set of transactions (same as the first popularity measure on `purchase_count` variable)

In [38]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
85123AY    2.375000
22423U     2.302832
85099BJ    2.295154
20725N     2.089710
47566Y     2.055690
21196B     2.000000
81953BF    2.000000
22720A     1.910256
21212D     1.831135
22197J     1.827930
84879M     1.805195
23298Q     1.793510
22383K     1.789894
22386V     1.752161
23442T     1.750000
20727V     1.716667
22960K     1.715517
47469B     1.714286
22469S     1.712766
23203B     1.710227
Name: purchase_count, dtype: float64

## 6. Collaborative Filtering Model

* In collaborative filtering, we would recommend items based on how similar users purchase items. For instance, if customer 1 and customer 2 bought similar items, e.g. 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.

* To define similarity across users, we use the following steps:
    1. Create a user-item matrix, where index values represent unique customer IDs and column values represent unique product IDs
    
    2. Create an item-to-item similarity matrix. The idea is to calculate how similar a product is to another product. There are a number of ways of calculating this. In steps 6.1 and 6.2, we use cosine and pearson similarity measure, respectively.  
    
        * To calculate similarity between products X and Y, look at all customers who have rated both these items. For example, both X and Y have been rated by customers 1 and 2. 
        * We then create two item-vectors, v1 for item X and v2 for item Y, in the user-space of (1, 2) and then find the `cosine` or `pearson` angle/distance between these vectors. A zero angle or overlapping vectors with cosine value of 1 means total similarity (or per user, across all items, there is same rating) and an angle of 90 degree would mean cosine of 0 or no similarity.
        
    3. For each customer, we then predict his likelihood to buy a product (or his purchase counts) for products that he had not bought. 
    
        * For our example, we will calculate rating for user 2 in the case of item Z (target item). To calculate this we weigh the just-calculated similarity-measure between the target item and other items that customer has already bought. The weighing factor is the purchase counts given by the user to items already bought by him. 
        * We then scale this weighted sum with the sum of similarity-measures so that the calculated rating remains within a predefined limits. Thus, the predicted rating for item Z for user 2 would be calculated using similarity measures.

* While I wrote python scripts for all the process including finding similarity using python scripts (which can be found in `scripts` folder, we can use `turicreate` library for now to capture different measures like using `cosine` and `pearson` distance, and evaluate the best model.

### 6.1. `Cosine` similarity
* Similarity is the cosine of the angle between the 2 vectors of the item vectors of A and B
* It is defined by the following formula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Closer the vectors, smaller will be the angle and larger the cosine

#### Using purchase count

In [39]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216824 observations with 971 users and 3766 items.

Data prepared in: 0.522481s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.89466s

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|   127269   |   22697A  |  0.2666719377040863 |  1   |
|   127269   |  85099BJ  |  0.2614979660511017 |  2   |
|   127269   |   22423U  |  0.2530737316608429 |  3   |
|   127269   |   22457C  |  0.2528485453128815 |  4   |
|   127269   |   20725N  |  0.2515132999420166 |  5   |
|   127269   |  85123AY  |  0.2514491057395935 |  6   |
|   127269   |   22197J  | 0.25118351221084595 |  7   |
|   127269   |   47566Y  | 0.24870201587677002 |  8   |
|   127269   |   21212D  | 0.24528229594230652 |  9   |
|   127269   |   22993E  | 0.24370922207832335 |  10  |
|   227268   |   22697A  |  0.2666719377040863 |  1   |
|   227268   |  85099BJ  |  0.2614979660511017 |  2   |
|   227268   |   22423U  |  0.2530737316608429 |  3   |
|   227268   |   22457C  |  0.2528485453128815 |  4   |
|   227268   |   20725N  |  0.2515132999420166 |

#### Using purchase dummy

In [40]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 968 users and 3757 items.

Data prepared in: 0.627002s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.09342s

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   127269   |  90129AJ  |  0.0  |  1   |
|   127269   |   21002B  |  0.0  |  2   |
|   127269   |   21936G  |  0.0  |  3   |
|   127269   |   22554P  |  0.0  |  4   |
|   127269   |   22319Q  |  0.0  |  5   |
|   127269   |   22470E  |  0.0  |  6   |
|   127269   |   90174N  |  0.0  |  7   |
|   127269   |   22666I  |  0.0  |  8   |
|   127269   |   22413V  |  0.0  |  9   |
|   127269   |   22294U  |  0.0  |  10  |
|   227268   |  90129AJ  |  0.0  |  1   |
|   227268   |   21002B  |  0.0  |  2   |
|   227268   |   21936G  |  0.0  |  3   |
|   227268   |   22554P  |  0.0  |  4   |
|   227268   |   22319Q  |  0.0  |  5   |
|   227268   |   22470E  |  0.0  |  6   |
|   227268   |   90174N  |  0.0  |  7   |
|   227268   |   22666I  |  0.0  |  8   |
|   227268   |   22413V  |  0.0  |  9   |
|   227268   |   22294U  |  0.0  |  10  |
|   327267   |  90129AJ  |  0.0  |

#### Using normalized purchase count

In [41]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 205580 observations with 971 users and 2494 items.

Data prepared in: 0.65724s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.0984s

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|   127269   |  85099BJ  | 0.050750924348831175 |  1   |
|   127269   |  85123AY  | 0.04938005089759827  |  2   |
|   127269   |   20725N  | 0.048133057355880735 |  3   |
|   127269   |   22423U  |  0.0472015392780304  |  4   |
|   127269   |   22383K  | 0.04320173144340515  |  5   |
|   127269   |   47566Y  | 0.041882468461990355 |  6   |
|   127269   |   22384W  | 0.041559535264968875 |  7   |
|   127269   |   22666I  | 0.04081364035606384  |  8   |
|   127269   |   22457C  | 0.03898935437202453  |  9   |
|   127269   |   22720A  | 0.03837816834449768  |  10  |
|   227268   |  85099BJ  | 0.050750924348831175 |  1   |
|   227268   |  85123AY  | 0.04938005089759827  |  2   |
|   227268   |   20725N  | 0.048133057355880735 |  3   |
|   227268   |   22423U  |  0.0472015392780304  |  4   |
|   227268   |   22383K  | 0.04

### 6.2. `Pearson` similarity
* Similarity is the pearson coefficient between the two vectors.
* It is defined by the following formula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Using purchase count

In [36]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216824 observations with 970 users and 3768 items.

Data prepared in: 0.85169s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 48.3525s

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|   127269   |   22423U  | 2.1809142635826353 |  1   |
|   127269   |  85123AY  | 2.1560581272728045 |  2   |
|   127269   |  85099BJ  | 2.082096195072575  |  3   |
|   127269   |   47566Y  | 1.9011910185267653 |  4   |
|   127269   |   20725N  | 1.8988365006292023 |  5   |
|   127269   |   22720A  | 1.8099884153363717 |  6   |
|   127269   |   84879M  | 1.7793387008220598 |  7   |
|   127269   |   22386V  | 1.7033959348635246 |  8   |
|   127269   |   21212D  | 1.6858641899865248 |  9   |
|   127269   |   22960K  | 1.666681425901662  |  10  |
|   227268   |   22423U  | 2.1809142635826353 |  1   |
|   227268   |  85123AY  | 2.1560581272728045 |  2   |
|   227268   |  85099BJ  | 2.082096195072575  |  3   |
|   227268   |   47566Y  | 1.9011910185267653 |  4   |
|   227268   |   20725N  | 1.8988365006292023 |  5   |
|   227268

#### Using purchase dummy

In [37]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 970 users and 3765 items.

Data prepared in: 0.845102s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 46.3728s

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   127269   |   22385P  |  0.0  |  1   |
|   127269   |   21154F  |  0.0  |  2   |
|   127269   |   22746G  |  0.0  |  3   |
|   127269   |   35953W  |  0.0  |  4   |
|   127269   |   21485W  |  0.0  |  5   |
|   127269   |   22089I  |  0.0  |  6   |
|   127269   |   22739W  |  0.0  |  7   |
|   127269   |   21843C  |  0.0  |  8   |
|   127269   |   23085W  |  0.0  |  9   |
|   127269   |   23296Z  |  0.0  |  10  |
|   227268   |   22385P  |  0.0  |  1   |
|   227268   |   21154F  |  0.0  |  2   |
|   227268   |   22746G  |  0.0  |  3   |
|   227268   |   35953W  |  0.0  |  4   |
|   227268   |   21485W  |  0.0  |  5   |
|   227268   |   22089I  |  0.0  |  6   |
|   227268   |   22739W  |  0.0  |  7   |
|   227268   |   21843C  |  0.0  |  8   |
|   227268   |   23085W  |  0.0  |  9   |
|   227268   |   23296Z  |  0.0  |  10  |
|   327267   |   22385P  |  0.0  |

#### Using normalized purchase count

In [38]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 205580 observations with 968 users and 2494 items.

Data prepared in: 0.84517s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 40.8697s

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|   127269   |   85089Y  |         1.0         |  1   |
|   127269   |   85189T  |         0.5         |  2   |
|   127269   |   23442T  |         0.5         |  3   |
|   127269   |   21196B  |         0.5         |  4   |
|   127269   |   84548X  |         0.5         |  5   |
|   127269   |   21816Y  |         0.5         |  6   |
|   127269   |  90065AP  | 0.33333333333333337 |  7   |
|   127269   |   21392H  | 0.33333333333333337 |  8   |
|   127269   |  84751BL  | 0.33333333333333337 |  9   |
|   127269   |   85146G  |  0.3333333333333333 |  10  |
|   227268   |   85089Y  |         1.0         |  1   |
|   227268   |   85189T  |         0.5         |  2   |
|   227268   |   23442T  |         0.5         |  3   |
|   227268   |   21196B  |         0.5         |  4   |
|   227268   |   84548X  |         0.5         |

#### Note
* In collaborative filtering above, we used two approaches: cosine and pearson distance. We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.
* We can see that the recommendations are different for each user. This suggests that personalization does exist. 
* But how good is this model compared to the baseline, and to each other? We need some means of evaluating a recommendation engine. Lets focus on that in the next section.

## 7. Model Evaluation
For evaluating recommendation engines, we can use the concept of precision-recall.

* RMSE (Root Mean Squared Errors)
    * Measures the error of predicted values
    * Lesser the RMSE value, better the recommendations
* Recall
    * What percentage of products that a user buys are actually recommended?
    * If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6
* Precision
    * Out of all the recommended items, how many the user actually liked?
    * If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8
    
* Why are both recall and precision important?
    * Consider a case where we recommend all products, so our customers will surely cover the items that they liked and bought. In this case, we have 100% recall! Does this mean our model is good?
    * We have to consider precision. If we recommend 300 items but user likes and buys only 3 of them, then precision is 0.1%! This very low precision indicates that the model is not great, despite their excellent recall.
    * So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

Lets compare all the models we have built based on precision-recall characteristics:

In [39]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Models on purchase counts

In [40]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts

Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |  0.3400211193241814 | 0.00709670566536427  |
|   2    | 0.31573389651531236 | 0.012485094107829919 |
|   3    |  0.2932066173882432 | 0.016527011110843037 |
|   4    | 0.28326293558606114 | 0.02138322476346206  |
|   5    | 0.27053854276663186 | 0.025556634139892703 |
|   6    | 0.25642379443857805 | 0.02868971780158148  |
|   7    | 0.24860461608085696 | 0.032857483731719726 |
|   8    | 0.24142027455121456 | 0.03566031695814584  |
|   9    |  0.2361844420978529 | 0.03942342485796968  |
|   10   | 0.23189017951425542 | 0.04305772010140779  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]



Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 56)"

Note that increasing the degree of parallelism also increases the memory footprint.


Overall RMSE: 0.5276032646172661

Per User RMSE (best)
+------------+----------------------+-------+
| customerId |         rmse         | count |
+------------+----------------------+-------+
|   442845   | 0.019641855032959583 |   2   |
+------------+----------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+------------+--------------------+-------+
| customerId |        rmse        | count |
+------------+--------------------+-------+
|   219177   | 1.3376068376068377 |   1   |
+------------+--------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+-----------+------+-------+
| productId | rmse | count |
+-----------+------+-------+
|   51008P  | 0.0  |   5   |
+-----------+------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+-----------+------+-------+
| productId | rmse | count |
+-----------+------+-------+
|   23442T  | 2.75 |   1   |
+-----------+------+-------+
[1 rows x 3 columns]

PROGRESS: Evaluate model Cosine Similarity on Pu

#### Models on purchase dummy

In [41]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy

Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    | 0.0063492063492063475 | 0.00013254765446349848 |
|   2    |  0.005291005291005293 | 0.00029388954098698327 |
|   3    |  0.00493827160493828  | 0.0003256960256741486  |
|   4    |  0.003703703703703706 | 0.0003256960256741486  |
|   5    | 0.0050793650793650785 | 0.0004270981869629655  |
|   6    |  0.005291005291005298 | 0.0005701913787454441  |
|   7    |  0.005442176870748298 | 0.0007106153715857964  |
|   8    |  0.005687830687830688 | 0.0008170054247531063  |
|   9    |  0.005878894767783658 | 0.0009339859348127636  |
|   10   |  0.00582010582010582  | 0.0010460911659635713  |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per Us

#### Models on normalized purchase frequency

In [42]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    |  0.002103049421661412 | 2.5192779530318973e-05 |
|   2    |  0.001051524710830706 | 2.5192779530318973e-05 |
|   3    | 0.0007010164738871369 | 2.5192779530318973e-05 |
|   4    | 0.0013144058885383803 | 8.394557534008266e-05  |
|   5    | 0.0012618296529968466 | 9.386561978188175e-05  |
|   6    | 0.0015772870662460587 | 0.00015460710402494357 |
|   7    | 0.0016523959741625363 | 0.00017592834240123443 |
|   8    | 0.0014458464773922184 | 0.00017592834240123445 |
|   9    | 0.0015188690267554635 | 0.00019846101477617793 |
|   10   | 0.0015772870662460572 | 0.00022455836246152588 |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.2

## 8. Model Selection
### 8.1. Evaluation summary
* Based on RMSE


    1. Popularity on purchase counts: 1.1111750034210488
    2. Cosine similarity on purchase counts: 1.9230643981653215
    3. Pearson similarity on purchase counts: 1.9231102838192284
    
    4. Popularity on purchase dummy: 0.9697374361161925
    5. Cosine similarity on purchase dummy: 0.9697509978436404
    6. Pearson similarity on purchase dummy: 0.9697745320187097
    
    7. Popularity on scaled purchase counts: 0.16230660626840343
    8. Cosine similarity on scaled purchase counts: 0.16229800354111104
    9. Pearson similarity on scaled purchase counts: 0.1622982668334026
    
* Based on Precision and Recall
![](../images/model_comparisons.png)


#### Notes

* Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.
* Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts. However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
* RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine.
Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

## 8. Final Output
* In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.
* We need to first rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [43]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_count', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 271031 observations with 972 users and 3810 items.

Data prepared in: 0.814579s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.83747s

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|   127269   |   20725N  | 0.33143950343132017 |  1   |
|   127269   |  85099BJ  |  0.3211128902435303 |  2   |
|   127269   |   22697A  |  0.3145043194293976 |  3   |
|   127269   |   20726C  |  0.3144213151931763 |  4   |
|   127269   |  85123AY  | 0.30583531856536866 |  5   |
|   127269   |   22423U  | 0.30275015115737913 |  6   |
|   127269   |   20719E  | 0.30248897910118105 |  7   |
|   127269   |   21212D  | 0.30203301191329956 |  8   |
|   127269   |   22383K  |  0.3003305244445801 |  9   |
|   127269   |   22411U  |  0.2998951661586762 |  10  |
|   227268   |   20725N  | 0.33143950343132017 |  1   |
|   227268   |  85099BJ  |  0.3211128902435303 |  2   |
|   227268   |   22697A  |  0.3145043194293976 |  3   |
|   227268   |   20726C  |  0.3144213151931763 |  4   |
|   227268   |  85123AY  | 0.30583531856536866 |

### 8.1. CSV output file

In [44]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head(10)

(6280, 4)


,customerId,productId,score,rank
0,127269,20725N,0.331440,1
1,127269,85099BJ,0.321113,2
2,127269,22697A,0.314504,3
3,127269,20726C,0.314421,4
4,127269,85123AY,0.305835,5
5,127269,22423U,0.302750,6
6,127269,20719E,0.302489,7
7,127269,21212D,0.302033,8
8,127269,22383K,0.300331,9
9,127269,22411U,0.299895,10


In [52]:
# df_rec1 = df_rec.to_dataframe()
import json
df2_unique = df_rec.drop_duplicates(subset='customerId')
final_recom = df2_unique.filter(['customerId',str('productId')])
final_recom = final_recom.rename(columns={'customerId': 'CustomerID', 'productId':'Items'})
final_recom['Items'] = final_recom['Items'].apply(lambda x: "'" + str(x) + "'")
final_recom['Items'] = '[' + final_recom['Items'].astype(str) + ']'

In [58]:
project.save_data(data=final_recom.to_csv(index=False,sep=','),file_name='recommended_purchase.csv',overwrite=True)

{'asset_id': 'c55f8ed8-9251-43fc-ab00-638411059644',
 'bucket_name': 'recommendationsystemforpurchaseda-donotdelete-pr-l0bytgn7jogfsi',
 'file_name': 'recommended_purchase.csv',
 'message': 'File saved to project storage.'}

In [54]:
print(final_recom.shape)
final_recom.head(20)

(628, 2)


,CustomerID,Items
0,127269,['20725N']
10,227268,['20725N']
20,327267,['20725N']
30,162738,['20725N']
40,252747,['20725N']
50,347247,['20725N']
60,442755,['20725N']
70,342756,['20725N']
80,57240,['20725N']
90,357237,['20725N']


In [46]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define a function to create a desired output

In [47]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('recommendation_output.csv')
    return df_output

In [48]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
rec_output=df_output.drop_duplicates()
project.save_data(data=rec_output.to_csv(index=False),file_name='recommendation_output.csv',overwrite=True)
print(df_output.shape)
df_output.head(12)

(628, 1)


,recommendedProducts
customerId,
1890,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
2700,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
3600,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
3690,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
4500,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
5490,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
8190,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
9090,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
13680,85089Y|85189T|23442T|21196B|84548X|21816Y|9006...


### 8.2. Customer recommendation function

In [49]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [50]:
customer_recomendation(19980)

recommendedProducts    85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
Name: 19980, dtype: object

In [51]:
customer_recomendation(15390)

recommendedProducts    85089Y|85189T|23442T|21196B|84548X|21816Y|9006...
Name: 15390, dtype: object

## Summary
In this exercise, we were able to traverse a step-by-step process for making recommendations to customers. We used Collaborative Filtering approaches with `cosine` and `pearson` measure and compare the models with our baseline popularity model. We also prepared three sets of data that include regular buying count, buying dummy, as well as normalized purchase frequency as our target variable. Using RMSE, precision and recall, we evaluated our models and observed the impact of personalization. Finally, we selected the Cosine approach in dummy purchase data. 